# BentoML vs FastAPI: Good-bye FastAPI For Machine Learning!
## Objective comparison between BentoML and FastAPI

### Notes

FastAPI vs BentoML

Similar Features
- Pedantic validation
- Swagger UI
- async
- Starlette

ML Differentiators
- Saving models
- Loading models
- Integrations with ML frameworks
- Batching inputs
    - https://docs.bentoml.org/en/latest/guides/batching.html
- Running models in separate processes
    - https://modelserving.com/blog/breaking-up-with-flask-amp-fastapi-why-ml-model-serving-requires-a-specialized-framework
- Automatic Containerizing
- Dependency management
- Monitoring out of the box
    - https://docs.bentoml.org/en/latest/guides/metrics.html
    - http://default-grafana.cloud-infra.bentoml.com/d/TJ3FhiG4z/bentodeployment?orgId=1&var-namespace=yatai&var-bento_deployment=credit-application&from=now-5m&to=now
- Open telemetry
- Automatic packaging
- Can combine fastapi and bentoml: https://docs.bentoml.org/en/latest/guides/server.html#bundle-asgi-app-e-g-fastapi
- GPU Serving
    - https://docs.bentoml.org/en/latest/guides/gpu.html#docker-images-options
- IODescriptors (ndarrays vs pydantic)
    - https://docs.bentoml.org/en/latest/reference/api_io_descriptors.html


### Motivation

### But what is an API?